In [229]:
import pickle

single_run = None
with open('data/single_run.pkl', 'rb') as rf:
    single_run = pickle.load(rf)

with open('data/n_run.pkl', 'rb') as rf:
    n_runs = pickle.load(rf)


In [213]:
import numpy as np
import time

In [215]:
chromo_set_1, score_set_1 = single_run

In [216]:
n_results = np.array(n_runs)

/var/folders/ms/33mxkwtn255b353qtr2c5m4r0000gn/T/ipykernel_60359/228490561.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  n_results = np.array(n_runs)


In [244]:
from n_run_evolutions import logmodel, X_train, Y_train, X_test, Y_test, accuracy_score
# from n_run_evolutions import all_models_score_table

In [235]:
def predicted_sentiment_ratio(y_test, predictions):
    positive_percent = np.count_nonzero(predictions==1)*100//len(predictions)
    negative_percent = np.count_nonzero(predictions==0)*100//len(predictions)
    # print(f"Y_test: pos/neg percentage ~ {np.count_nonzero(y_test==1)*100//len(y_test)}, {np.count_nonzero(y_test==0)*100//len(y_test)}")
    return positive_percent, negative_percent

In [236]:
def test_accuracy(selected_genes):
    st = time.time()
    logmodel.fit(X_train[:, selected_genes], Y_train)
    et = time.time()

    predictions = logmodel.predict(X_test[:, selected_genes])
    ratio = predicted_sentiment_ratio(Y_test, predictions)
    return accuracy_score(Y_test, predictions), et-st, ratio

# Collect Common indexes from res of single run evolution


In [237]:
def pick_top_n_genes_1_gen(n=-1):
    common_index_count = dict()
    chromo_len_in_each_gen = []
    for chromo in chromo_set_1:
        chromo_len_in_each_gen.append(np.count_nonzero(chromo == 1))
        for i in range(len(chromo)):
            if chromo[i] == 1:
                common_index_count[i] = common_index_count.setdefault(i, 0) + 1

    if n == -1:
        n = len(common_index_count)

    sorted_common_index = np.array(sorted(
        common_index_count.items(), key=lambda x: x[1], reverse=True)[:n])
    return sorted_common_index[:, 0], len(sorted_common_index), np.array(chromo_len_in_each_gen)

In [204]:
X_train.shape

(750, 1553)

In [239]:
from tabulate import tabulate

genes = [100, 150, 200, 250, -1]
accuracy = []
time_to_train = []
sentiment_ratio = []
for i in genes:
    top_n_genes, common_count, chromo_len = pick_top_n_genes_1_gen(i)
    acc, ttt, sr = test_accuracy(top_n_genes)
    accuracy.append(acc)
    time_to_train.append(ttt)
    sentiment_ratio.append(sr)


table_data = {'Genes': genes, 
        'Accuracy': accuracy, 
        'Time Taken to Train': time_to_train, 
        'Sentiment Ratio(p/n)': sentiment_ratio
        }

print('Actual chromosome length in X_Test: ', X_test.shape[1])
print('RandomForest with full length: ', 0.800, 1.506469)
print('Total Common genes count ~(-1): ', common_count)
print(chromo_len)
print(tabulate(table_data, headers='keys', tablefmt="simple_grid"))

Actual chromosome length in X_Test:  1553
RandomForest with full length:  0.8 1.506469
Total Common genes count ~(-1):  1377
[100 312 480 318 342 342 342 342 343 467 333 346 344 344 344 378 504 533
 533 533 482 482 482 482 467 467 467 467 467 467]
┌─────────┬────────────┬───────────────────────┬────────────────────────┐
│   Genes │   Accuracy │   Time Taken to Train │ Sentiment Ratio(p/n)   │
├─────────┼────────────┼───────────────────────┼────────────────────────┤
│     100 │      0.728 │              0.940133 │ (37, 62)               │
├─────────┼────────────┼───────────────────────┼────────────────────────┤
│     150 │      0.764 │              0.734061 │ (40, 60)               │
├─────────┼────────────┼───────────────────────┼────────────────────────┤
│     200 │      0.772 │              0.421631 │ (40, 60)               │
├─────────┼────────────┼───────────────────────┼────────────────────────┤
│     250 │      0.752 │              1.34413  │ (44, 55)               │
├─────────┼─

# Collect Common indexes from res of n run evolution


In [240]:
def pick_top_n_genes_n_gen(n=-1):
    chromo_len_best_gen = []
    n_best_gen = []
    common_index_n_runs = dict()

    for run in n_results:
        best_gen = np.array(run[:, -1])
        n_best_gen.append(best_gen)
        chromo_len_best_gen.append(np.count_nonzero(best_gen[0] == 1))
        # print(chromo_len_best_gen)
        # print(best_gen[0])
        for i in range(len(best_gen[0])):
            if best_gen[0][i] == 1:
                common_index_n_runs[i] = common_index_n_runs.setdefault(i, 0) + 1
    if n == -1:
        n = len(common_index_n_runs)

    sorted_common_index_n_runs = np.array(
        sorted(common_index_n_runs.items(), key=lambda x: x[1], reverse=True)[:n])
    return sorted_common_index_n_runs[:, 0], len(sorted_common_index_n_runs), chromo_len_best_gen

In [241]:
from tabulate import tabulate

genes = [100, 150, 200, 250, -1]
accuracy = []
time_to_train = []
sentiment_ratio = []
for i in genes:
    top_n_genes, common_count, chromo_len = pick_top_n_genes_n_gen(i)
    acc, ttt, sr = test_accuracy(top_n_genes)
    accuracy.append(acc)
    time_to_train.append(ttt)
    sentiment_ratio.append(sr)


table_data = {'Genes': genes, 
        'Accuracy': accuracy, 
        'Time Taken to Train': time_to_train, 
        'Sentiment Ratio(p/n)': sentiment_ratio
        }

print('Actual chromosome length in X_Test: ', X_test.shape[1])
print('RandomForest with full length: ', 0.800, 1.506469)
print('Total Common genes count ~(-1): ', common_count)
print(chromo_len)
print(tabulate(table_data, headers='keys', tablefmt="simple_grid"))

Actual chromosome length in X_Test:  1553
RandomForest with full length:  0.8 1.506469
Total Common genes count ~(-1):  1553
[612, 506, 369, 542, 613, 643, 510, 580, 436, 415, 535, 476, 585, 422, 434, 448, 590, 536, 468, 596, 438, 493, 538, 450, 463, 606, 468, 477, 498, 528]
┌─────────┬────────────┬───────────────────────┬────────────────────────┐
│   Genes │   Accuracy │   Time Taken to Train │ Sentiment Ratio(p/n)   │
├─────────┼────────────┼───────────────────────┼────────────────────────┤
│     100 │      0.796 │              0.454911 │ (37, 62)               │
├─────────┼────────────┼───────────────────────┼────────────────────────┤
│     150 │      0.808 │              0.427708 │ (38, 61)               │
├─────────┼────────────┼───────────────────────┼────────────────────────┤
│     200 │      0.836 │              0.528511 │ (41, 58)               │
├─────────┼────────────┼───────────────────────┼────────────────────────┤
│     250 │      0.828 │              0.355206 │ (43, 56) 